In [1]:
#using TaylorSeries
include("UndrivenHeliumParse.jl")
#include("Integ.jl")
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 36
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-22
const steps = 500_000;

## Revisamos compatibilidad hamiltoniano vs ecs de movimiento

In [ ]:
#@time dnorm= errHam2D(10_000);
plot(eachindex(dnorm), dnorm, title = "Compatibility Hamiltonian vs Eqs of motion")
xaxis!("Machine epsilons")

# Incluimos funciones para obtener secciones de Poincaré

In [2]:
@taylorize_ode function undrivenHelium2Dp!(τ, q, dq)
      
    #Cantidades auxiliares
    Q₁² = q[1]^2 + q[2]^2
    Q₂² = q[3]^2 + q[4]^2
    P₁² = q[5]^2 + q[6]^2
    P₂² = q[7]^2 + q[8]^2
    t = Q₁²*Q₂²
    f₁ = (q[1]^2 - q[2]^2) - (q[3]^2 - q[4]^2)
    f₂ = 2(q[1]*q[2]) - 2(q[3]*q[4])
    fs = f₁^2 + f₂^2
    nf = sqrt(fs)
    c1 = 1 + 1/nf
    nf³ = nf^3
    factor1 = t/nf³
    s1 = -0.125P₂² + (Z - Q₂²*c1)
    s2 = factor1*f₁
    s3 = -0.125P₁² + (Z - Q₁²*c1)
    #@show c1
    dq[1] = 0.25(Q₂²*q[5])
    dq[2] = 0.25(Q₂²*q[6])
    dq[3] = 0.25(Q₁²*q[7])
    dq[4] = 0.25(Q₁²*q[8])
    dq[5] = 2q[1]*(s1 + s2) + (factor1*f₂)*q[2]
    dq[6] = 2q[2]*(s1 - s2) + (factor1*f₂)*q[1]
    dq[7] = 2q[3]*(s3 - s2) - (factor1*f₂)*q[4]
    dq[8] = 2q[4]*(s3 + s2) - (factor1*f₂)*q[3]
    return nothing
end

In [3]:
include("Integ.jl")

taylorintegps (generic function with 2 methods)

In [ ]:
#condini2D(q₁x::T, q₁y::T, p₁x::T, p₁y::T, P₂y::T) where T<:Number in Main at /home/daniel/Documents/2017-2/Tesis/UndrivenHelium/TaylorIntegration/UndrivenHeliumParse.jl:110()
c1 = condini2D(5.8, 0, 0., 0.0, 0.0)

In [ ]:
regHam2D(c1)

In [ ]:
t1p, x1p = taylorinteg(undrivenHelium2Dp!, c1, t0, 1.0, order, abs_tol, maxsteps=100)
@time t1p, x1p = taylorinteg(undrivenHelium2Dp!, c1, t0, 120.0, order, abs_tol, maxsteps=10_000);

In [ ]:
t1, x1 = taylorinteg(undrivenHelium2D!, c1, t0, 1.0, order, abs_tol, maxsteps=100)
@time t1, x1 = taylorinteg(undrivenHelium2D!, c1, t0, 120.0, order, abs_tol, maxsteps=10_000);

In [ ]:
length(t1p), length(t1)

In [ ]:
norm(x1-x1p, Inf)

In [ ]:
ene1 = zeros(t1)
ene2 = zeros(t1)
for j in eachindex(ene1)
    ene1[j] = abs(regHam2D(x1p[j,:]))/eps()
end
for j in eachindex(ene2)
    ene2[j] = abs(regHam2D(x1[j,:]))/eps()
end

In [ ]:
plot(t1, ene1, title =L"Absolute Error of Energy $q_1=2$", color = :black)
#plot!(t2, ene2, title =L"Absolute Error of Energy $q_1=2$",color=:red, label=L"Cálculo con Sección de Poincaré")
xaxis!(L"t")
yaxis!(L"\delta E")


In [ ]:
plot(t1, norm.(x1[:,1]), title =L"Absolute Error of Energy $q_1=2$", color = :black)

In [ ]:
minimum(norm.(x1[:,1] - x1[:,3]))

In [ ]:
plot(t2, ene2, title =L"Absolute Error of Energy $q_1=2$")
xlabel!(L"t")
ylabel!(L"\delta E")

In [ ]:
maximum(ene1)